In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from explorers.PPO_explorer import PPO_explorer

from utils.sequence_utils import generate_random_mutant, generate_random_sequences

## 1. Building oracles for landscapes

In [38]:
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Ground_truth_oracles.TF_binding_landscape_models import *

In [39]:
landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])

landscape_generator = landscape_constructor.generate_from_loaded_landscapes()

#take one landscape oracle from the generator
landscape_object=next(landscape_generator)

print(landscape_object['landscape_id'])
print(landscape_object['starting_seqs'])
landscape_oracle = landscape_object['landscape_oracle']

wt = landscape_object['starting_seqs']['TF0']

1 TF landscapes loaded.
SIX6_REF_R1
{'TF0': 'TTAATTAA', 'TF1': 'GCTCGAGC', 'TF2': 'GCGCGCGC', 'TF3': 'TGCGCGCC', 'TF4': 'ATATAGCC', 'TF5': 'GTTTGGTA', 'TF6': 'ATTATGTT', 'TF7': 'CAGTTTTT', 'TF8': 'AAAAATTT', 'TF9': 'AAAAACGC', 'TF10': 'GTTGTTTT', 'TF11': 'TGCTTTTT', 'TF12': 'AAAGATAG', 'TF13': 'CCTTCTTT', 'TF14': 'AAAGAGAG'}


## 2. Building a model of the landscape

In [90]:
from utils.model_architectures import CNNa
from models.Noisy_models.Neural_network_models import NN_model

alphabet = "ATGC"

training_sequences = list(set([generate_random_mutant(wt, 0.2, alphabet) for i in range(0, 5)]))
cnn_architecture=CNNa(len(wt), alphabet=alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

## 3. Exploring the landscape with a model

In [91]:
# from explorers.base_explorer import Random_explorer

# mutation_rate = 0.05
# explorer = Random_explorer(mutation_rate,
#                            batch_size=100,
#                            alphabet=alphabet,
#                            virtual_screen=10)
# explorer.debug = True # don't save results
# explorer.set_model(noisy_nn_model)

In [92]:
# num_of_rounds = 5
# explorer.run(num_of_rounds, verbose=True)

### PPO explorer

In [104]:
ppo_explorer = PPO_explorer(
    alphabet=alphabet)
ppo_explorer.debug = True
ppo_explorer.set_model(noisy_nn_model)

In [105]:
ppo_explorer.pretrain_agent()

Episodes: 0/100
Number of measured sequences: 54
New top sequence: (0.6781429101554447, 'TTAATTAA', 5)


In [ ]:
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

num_of_rounds = 5
ppo_explorer.run(num_of_rounds, verbose=True)

round: 0, cost: 5, evals: 98, top: 0.6781429101554447
54


In [103]:
noisy_nn_model.evals

0